In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('tree').getOrCreate()

In [3]:
from pyspark.ml.classification import (DecisionTreeClassifier,RandomForestClassifier,GBTClassifier)
from pyspark.ml import Pipeline

In [4]:
df = spark.read.format('libsvm').load('/FileStore/tables/sample_libsvm_data.txt')

In [5]:
df.show(5)

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows

In [6]:
train,test = df.randomSplit([0.7,0.3])

In [7]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees=100)
gbt = GBTClassifier()

In [8]:
dtc_model = dtc.fit(train)
rfc_model = rfc.fit(train)
gbt_model = gbt.fit(train)

In [9]:
dtc_pred = dtc_model.transform(test)
rfc_pred = rfc_model.transform(test)
gbt_pred = gbt_model.transform(test)

In [10]:
dtc_pred.show(5)

+-----+--------------------+-------------+-----------+----------+
label| features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
 0.0|(692,[95,96,97,12...| [29.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[122,123,124...| [29.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[123,124,125...| [29.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[126,127,128...| [29.0,0.0]| [1.0,0.0]| 0.0|
 0.0|(692,[126,127,128...| [29.0,0.0]| [1.0,0.0]| 0.0|
+-----+--------------------+-------------+-----------+----------+
only showing top 5 rows

In [11]:
# In spark,we can use MulticlassClassificationEvaluator in a binary classification problem
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [12]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [13]:
print('DTC ACCURACY:')
acc_eval.evaluate(dtc_pred)

DTC ACCURACY:
 Out[ 32 ]: 0.9655172413793104

In [14]:
print('RFC ACCURACY:')
acc_eval.evaluate(rfc_pred)

RFC ACCURACY:
 Out[ 30 ]: 1.0

In [15]:
print('GBT ACCURACY:')
acc_eval.evaluate(gbt_pred)

GBT ACCURACY:
 Out[ 31 ]: 0.9655172413793104

In [16]:
# feature importance
rfc_model.featureImportances

Out[ 34 ]: SparseVector(692, {126: 0.0004, 151: 0.0044, 152: 0.0008, 156: 0.0005, 158: 0.0004, 178: 0.0004, 179: 0.0003, 182: 0.0014, 186: 0.0007, 188: 0.0005, 204: 0.0006, 209: 0.0001, 216: 0.0005, 231: 0.0004, 234: 0.0068, 239: 0.0003, 243: 0.0152, 244: 0.0026, 245: 0.0054, 262: 0.0013, 263: 0.015, 264: 0.0004, 268: 0.0006, 272: 0.0165, 273: 0.016, 289: 0.0025, 297: 0.0011, 299: 0.0067, 300: 0.0264, 301: 0.0086, 304: 0.0011, 313: 0.0006, 314: 0.0032, 322: 0.0011, 323: 0.0005, 325: 0.0011, 328: 0.0097, 330: 0.0061, 332: 0.0019, 345: 0.0007, 346: 0.0073, 350: 0.0167, 351: 0.0418, 352: 0.0023, 354: 0.0008, 355: 0.0061, 356: 0.0059, 357: 0.0227, 359: 0.0005, 371: 0.0011, 373: 0.024, 374: 0.0064, 377: 0.0165, 378: 0.0341, 379: 0.0022, 383: 0.0011, 384: 0.0011, 385: 0.0082, 386: 0.0065, 398: 0.0019, 401: 0.0116, 402: 0.0027, 405: 0.0102, 406: 0.0422, 407: 0.0022, 413: 0.0006, 415: 0.0006, 416: 0.0025, 427: 0.0075, 428: 0.0126, 429: 0.0079, 430: 0.0023, 431: 0.002, 433: 0.0195, 434: 0.0468, 435: 0.0005, 438: 0.001, 439: 0.0027, 440: 0.0248, 442: 0.0064, 443: 0.0038, 455: 0.0451, 456: 0.0156, 457: 0.0022, 461: 0.0073, 462: 0.0256, 463: 0.0253, 464: 0.0006, 467: 0.0034, 468: 0.0097, 482: 0.0021, 483: 0.0088, 484: 0.0012, 485: 0.0071, 489: 0.0005, 490: 0.0523, 492: 0.0003, 495: 0.0003, 496: 0.0067, 497: 0.0069, 498: 0.0013, 511: 0.0206, 512: 0.0326, 516: 0.0006, 517: 0.0071, 518: 0.0065, 519: 0.0013, 521: 0.0011, 522: 0.0056, 523: 0.0008, 524: 0.0004, 528: 0.0029, 539: 0.0193, 540: 0.0583, 544: 0.0005, 545: 0.0082, 550: 0.0005, 554: 0.0024, 567: 0.0073, 568: 0.0003, 574: 0.0003, 576: 0.0005, 578: 0.0074, 579: 0.0004, 580: 0.001, 603: 0.001, 604: 0.0043, 627: 0.0013, 630: 0.0008, 631: 0.0006, 651: 0.0003, 656: 0.0007, 658: 0.0007, 684: 0.0013})